### Ingest races.csv file

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Step 1 - Read the CSV file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", StringType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True)])

In [0]:
races_df = spark.read \
    .schema(races_schema) \
    .option("header", True) \
    .csv(f'{raw_folder_path}/{v_file_date}/races.csv')

#### Step 2 - Add Additional Columns Required

In [0]:
from pyspark.sql.functions import col, lit, concat, to_timestamp

In [0]:
races_columnAdded_df = add_ingestion_date(races_df.withColumn("race_timestamp",to_timestamp(concat(col('date'), lit(' '), col('time')),'yyyy-MM-dd HH:mm:ss')) \
.withColumn("data_source",lit(v_data_source)) \
.withColumn("file_date",lit(v_file_date)))

#### Step 3 - Select only the required columns and rename them

In [0]:
from pyspark.sql.functions import col

In [0]:
races_selected_df = races_columnAdded_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), col('round'), col('circuitId').alias('circuit_id'), col('name'), col('race_timestamp'), col('ingestion_date'), col('data_source'), col('file_date'))

#### Step 5 - Write dataframe as parquet to processed folder in datalake

In [0]:
races_selected_df.write.mode("overwrite").partitionBy('race_year').format("delta").saveAsTable("f1_processed.races")

In [0]:
dbutils.notebook.exit("Success")